In [ ]:
# default_exp datasets.sample_file

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
from corradin_ovp_utils.catalog import test_data_catalog, conf_test_data_catalog

In [ ]:
#export
from typing import Any, Dict, List, Optional, Literal, Union
from pydantic import BaseModel
from pathlib import Path, PosixPath
from fastcore.meta import delegates
import pandas as pd
from copy import deepcopy

# from enum import Enum
# import numpy as np
# from kedro.io import AbstractVersionedDataSet

# from pydantic.dataclasses import dataclass
# from dataclasses import InitVar, asdict
# from corradin_ovp_utils.datasets import genetic_datasets
# from kedro.io.core import (
#     AbstractVersionedDataSet,
#     DataSetError,
#     Version,
#     get_filepath_str,
#     get_protocol_and_path,
# )
# import fsspec


# from types import SimpleNamespace

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#export

class SampleFileFormat(BaseModel):
    filepath: Union[str, Path]
    pandas_args: Dict[str, Any]
    sample_id_col: str
    cov_cols: List[str]
    missing_col: Optional[str]
    
    
    @delegates(pd.read_csv)
    def load(self, **kwargs):
        df = pd.read_csv(filepath_or_buffer = self.filepath, **self.load_args, **kwargs)
        return df
    
    @property
    def load_args(self):
        load_args = deepcopy(self.pandas_args)
        load_args["index_col"] = self.sample_id_col
        return load_args

In [ ]:
test_sample_dataset = test_data_catalog.load("sample_file")
test_sample_dataset.files.case.load()

,ID_1,missing,sex,case
ID_2,,,,
WTCCCT473540,95300_D10,0,2,1
WTCCCT473530,95300_D11,0,1,1
WTCCCT473555,95300_D12,0,1,1
WTCCCT473426,95300_E01,0,2,1
WTCCCT473489,95300_E02,0,1,1
...,...,...,...,...
WTCCCT473455,116733_C03,0,1,1
WTCCCT473479,116733_D03,0,2,1
WTCCCT473432,116733_E03,0,2,1


In [ ]:
test_sample_dataset.files

namespace(case=SampleFileFormat(filepath=Path('data/test_data/sample_file/MS_impute2_ALL_sample_out.tsv'), pandas_args={'sep': ' ', 'skiprows': [1]}, sample_id_col='ID_2', cov_cols=['sex'], missing_col='missing'),
          control=SampleFileFormat(filepath=Path('data/test_data/sample_file/ALL_controls_58C_NBS_WTC2_impute2_sample_out.tsv'), pandas_args={'sep': ' ', 'skiprows': [1]}, sample_id_col='ID_2', cov_cols=['sex'], missing_col='missing'))

In [ ]:
test_sample_dataset.files.control.load()

,ID_1,missing,plate,sex,case
ID_2,,,,,
WTCCCT443025,74717_A02,0,74717,2,0
WTCCCT443065,74717_A05,0,74717,2,0
WTCCCT443063,74717_A06,0,74717,2,0
WTCCCT443026,74717_B02,0,74717,2,0
WTCCCT443066,74717_B05,0,74717,2,0
...,...,...,...,...,...
WS574632,101916_C07,0,101916,1,0
WS574661,101916_D07,0,101916,1,0
BLOOD294452,101806_F08,0,101806,1,0


In [ ]:
index_case = test_sample_dataset.files.case.load().index
index_control = test_sample_dataset.files.control.load().index
assert index_case.is_unique
assert index_control.is_unique
assert set(index_case) & set(index_control) == set() #no overlap between cases and controls